# 02 - Data from the Web

In [ ]:
# Needed imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import difflib
import seaborn as sns
import numpy as np
%matplotlib inline

## Some helper functions we need

These functions will be used throughout the homework.

In [ ]:
# Parses a string to an integer, removing invalid characters
def parseInt(numStr):
    cleaned = [x for x in numStr if x.isdigit()]
    return int("".join(cleaned))

# Parses a string to a float, removing invalid characters
def parseDecimal(numStr):
    cleaned = [x for x in numStr if x.isdigit() or x == '.']
    return float("".join(cleaned))

## Task 1


### a. Load data
- Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). We can obtain this information by parsing the JSON file from the QS website.

In [ ]:
r = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt')
qs_dataset = r.json()['data']

In [ ]:
# Sample record
qs_dataset[0]

- For each university, we extract: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Since the detailed information (faculty and students) is not avaiable in the main JSON file, we gather it from the respective page of each university by parsing the HTML content.
 - Note: the values of students and faculty members may be missing in the details page. In this case, we set them to 0.

In [ ]:
results = []
for university in qs_dataset[:200]:
    url = 'https://www.topuniversities.com' + university['url']
    details_html = requests.get(url)
    soup = BeautifulSoup(details_html.text, 'html.parser')

    def parseAttribute(className):
        attr = soup.find('div', class_=className)
        if attr is not None:
            return parseInt(attr.find('div', class_='number').text)
        else:
            return 0 # Missing value
        
    results.append({
            'name': university['title'],
            'rank': parseInt(university['rank_display']),
            'country': university['country'],
            'region': university['region'],
            'faculty_international': parseAttribute('inter faculty'),
            'faculty_total': parseAttribute('total faculty'),
            'students_international': parseAttribute('total inter'),
            'students_total': parseAttribute('total student')
        })

- Convert the resulting dataset into a DataFrame and store it as a file (for reusing it later without parsing the content again).

In [ ]:
qs_df = pd.DataFrame.from_dict(results).set_index('name')
qs_df.to_pickle('qs_dataset')

In [ ]:
qs_df = pd.read_pickle('qs_dataset')
qs_df.head() # Show first elements

### Remarks
- There is no university with rank #198. As a result, the interval of the ranking is [1,201]. This is an error in the QS ranking.

### b. Add the columns 'faculty_students_ratio' and 'international_ratio'

**'faculty_students_ratio'** represents the faculty-to-students ratio.

**'international_ratio'** represents the fraction of international students. 

In [ ]:
qs_df['faculty_students_ratio'] = qs_df['faculty_total']/qs_df['students_total']
qs_df['international_ratio'] = qs_df['students_international']/qs_df['students_total']
# For this analysis, we keep only the columns in which we are interested
qs_df_analysis = qs_df.loc[:, ['region', 'country','rank', 'faculty_students_ratio', 'international_ratio']]
qs_df_analysis.head()

### c. Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- By sorting along these columns we can respectively find the best universities in terms of (a) ratio between faculty members and students and (b) ratio of international students.

In [ ]:
sorted_ = qs_df_analysis.sort_values('faculty_students_ratio', ascending=False).reset_index()
sorted_.index = range(1, len(sorted_) + 1) # Start from index = 1
sorted_.head()

Caltech is the university with the highest faculty/students ratio. It also appears that the best universities in this group are in the top 20 of the QS ranking.

In [ ]:
sorted_ = qs_df_analysis.sort_values('international_ratio', ascending=False).reset_index()
sorted_.index = range(1, len(sorted_) + 1) # Start from index = 1
sorted_.head()

The London School of Economics and Political Science is the most international university, followed by EPFL.

### d. Answer the previous question aggregating the data by (c) country and (d) region.
- We answer the same question aggregating the data by (c) **country** and (d) **region**. For each group, we show the average of the students-to-faculty ratio and international students ratio. Note that every table reports the mean, the standard error of the mean (which is reported in the bar plot as the confidence line), and the count (number of universities in top 200). These statistics will come handy in our observations.

In [ ]:
# We define a helper function for aggregating the data and drawing the graph.
# This function will be used for the analysis of both QS and THE datasets.
# In our case, 'grouping_col' will be either 'country' or 'region,
# and 'value_col' will be either 'faculty_students_ratio' or 'international_ratio'.
def aggregate_analyze(df, grouping_col, value_col, title=""):
    '''
    Arguments:
    -- df: the DataFrame to process
    -- grouping_col: the column by which the data will be grouped
    -- value_col: the variabile to average within each group
    '''

    grouped = df.groupby([grouping_col]).agg({value_col: ['mean', 'sem', 'count']})\
        .sort_values([(value_col, 'mean')], ascending=False)
    
    grouped_with_rank = grouped.reset_index().fillna('-')
    grouped_with_rank.index = range(1, len(grouped) + 1) # Start from index = 1
    
    # Plot
    ax = grouped[(value_col, 'mean')].plot.bar(title = title, figsize=(10,5), yerr=grouped[(value_col, 'sem')])
    #ax.legend_.remove()
    ax.set_ylabel(value_col.replace('_', ' ').capitalize())
    ax.set_xlabel(grouping_col.replace('_', ' ').capitalize())
    
    return grouped_with_rank

In [ ]:
aggregate_analyze(qs_df_analysis, 'country', 'faculty_students_ratio', title="Average of faculty/student ratio by country")

In [ ]:
aggregate_analyze(qs_df_analysis, 'country', 'international_ratio', title="Average of international students ratio by country")

In [ ]:
aggregate_analyze(qs_df_analysis, 'region', 'faculty_students_ratio', title="Average of faculty/students ratio by region")

In [ ]:
aggregate_analyze(qs_df_analysis, 'region', 'international_ratio', title="Average of international students ratio by country")

### Remarks
In our opinion, there is some bias in these statistics. As can be seen from the confidence metrics, some countries/regions have very few universities in the top 200, and therefore the sample size is not big enough for obtaining a reliable result. For example, Russia is reportedly the state with the highest student/faculty ratio (0.22), but there is only one university from Russia. Hence, this result is not very meaningful.

## Task 2
We now obtain the 200 top-ranking universities from www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)) and repeat the previous analysis. Similarly as before, we found a JSON file that contains the list of universities in the main web page.

In [ ]:
r = requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
the_dataset = r.json()['data']

In [ ]:
the_dataset[0] # Show an example

There are some small differences between the QS and THE datasets, and we need to align their variables as best as possible.
- THE reports the inverse of the faculty-to-students ratio.
- Only the percentage of international students is provided. We can easily compute the absolute number by multiplying it with the total number of students.
- The number of faculty members is not provided. We can compute it from the faculty/students ratio.
- There is no information about the number/percentage of international faculty. We cannot infer this data.
- The region is not given, but we can compute it easily by matching each country with the QS dataset.

In [ ]:
results = []
for university in the_dataset[:200]:
    results.append({
            'name': university['name'],
            'rank': parseInt(university['rank']),
            'country': university['location'],
            'faculty_students_ratio': 1 / parseDecimal(university['stats_student_staff_ratio']),
            'international_ratio': parseDecimal(university['stats_pc_intl_students']) / 100,
            'students_total': parseInt(university['stats_number_students'])
        })

the_df = pd.DataFrame.from_dict(results).set_index('name')

# Compute derived attributes (where applicable)
the_df['students_international'] = (the_df['students_total'] * the_df['international_ratio']).astype('int')
the_df['faculty_total'] = (the_df['students_total'] * the_df['faculty_students_ratio']).astype('int')

In [ ]:
# Map countries to regions using the QS dataset
mapping = qs_df[['country', 'region']].set_index('country').to_dict()['region']

# Manually add missing mappings that are not present in the QS dataset
mapping['Luxembourg'] = 'Europe'
mapping['Russian Federation'] = 'Europe'

the_df['region'] = the_df['country'].replace(mapping)
the_df['country'].replace({'Russian Federation': 'Russia'}, inplace=True)

# For this analysis keep only the columns in which we are interested
the_df_analysis = the_df.loc[:, ['region', 'rank', 'country', 'faculty_students_ratio', 'international_ratio']]
the_df_analysis.head()

### Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- As before, by sorting along these columns we can respectively find the best universities in terms of (a) ratio between faculty members and students and (b) ratio of international students.

In [ ]:
sorted_ = the_df_analysis.sort_values('faculty_students_ratio', ascending=False).reset_index()
sorted_.index = range(1, len(sorted_) + 1) # start index from 1
sorted_.head()

In [ ]:
sorted_ = the_df_analysis.sort_values('international_ratio', ascending=False).reset_index()
sorted_.index = range(1, len(sorted_) + 1) # start index from 1
sorted_.head()

### Answer the previous question aggregating the data by (c) country and (d) region.
- We answer the same question aggregating the data by (c) **country** and (d) **region**. For each group, we show the average of student-to-faculty ratio and international students ratio.

In [ ]:
aggregate_analyze(the_df_analysis, 'country', 'faculty_students_ratio', title="Average of faculty/student ratio by country")

In [ ]:
aggregate_analyze(the_df_analysis, 'country', 'international_ratio', title="Average of international students ratio by country")

In [ ]:
aggregate_analyze(the_df_analysis, 'region', 'faculty_students_ratio', title="Average of faculty/student ratio by region")

In [ ]:
aggregate_analyze(the_df_analysis, 'region', 'international_ratio', title="Average of international students ratio by region")

### Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.
We implemented a fuzzy join algorithm for merging the two rankings. Since the names of the universities may differ slightly, it is necessary to use string matching algorithm based on a similarity score.
Our algorithm works as follows:
- For each university in the first ranking (QS), we find the corresponding university in the second ranking (THE) that minimizes the edit distance between the two strings. There are several metrics that could be used: we used the one that is implemented in Python's difflib (a variant of the Ratcliff-Obershelp algorithm). Other choices are possible (e.g. Levenshtein distance). The match is accepted only if all these conditions are true:
 - The countries of the two matches correspond.
 - The edit distance is below a certain threshold.
 - If we match university A to university B, the match in the opposite direction is consistent (i.e. we match B to A).
 
Of course, it is not possible to match all universities. Some universities that are present in the QS ranking may not be present in the THE ranking, and vice versa.

In [ ]:
not_matched_counter = 0

qs_df_ = qs_df.copy().reset_index()
the_df_ = the_df.copy().reset_index()

mapping = qs_df_['name'].copy()
for i in range(len(qs_df_)):
    def match():
        tolerance = 0.6 # Threshold that regulates the sensitivity of the algorithm
        res = difflib.get_close_matches(qs_df_['name'][i], the_df_['name'], 1, tolerance)
        if len(res) > 0:
            matching_country = the_df_['country'][the_df_['name'] == res[0]]
            if len(matching_country) == 1 and matching_country.values[0] == qs_df_['country'][i]:
                back_res = difflib.get_close_matches(res[0], qs_df_['name'], 1, tolerance)
                if len(back_res) > 0 and qs_df_['name'][i] == back_res[0]:
                    mapping[i] = res[0]
                    return True
        return False
    if not match():
        not_matched_counter += 1

qs_df_['name'] = mapping
print(not_matched_counter, 'universities were not matched.')
merged_df = qs_df_.merge(the_df_, on='name', how='inner', suffixes=('_qs', '_the'))

# Clean the table (only one column for the region and the country)
merged_df["region"] = merged_df.region_qs
merged_df["country"] = merged_df.country_qs
merged_df.drop(['region_qs', 'region_the', 'country_qs', 'country_the'], axis = 1, inplace = True)

In [ ]:
# Represent the columns in a cleaner way
cleaned = merged_df.copy()[['name', 'country', 'region', 'faculty_international', 'rank_qs', 'rank_the', "faculty_total_qs", "faculty_total_the",\
  'faculty_students_ratio_qs', 'faculty_students_ratio_the', 'international_ratio_qs', 'international_ratio_the',\
       'students_international_qs', 'students_international_the', 'students_total_qs', 'students_total_the']]
col_level1 = ['', 'rank', 'faculty_total', 'faculty_students_ratio', 'international_ratio', 'students_international', 'students_total']
col_level2 = ['name', 'country', 'region', 'faculty_international_qs'] + ["qs", "the"]*(len(col_level1)-1)#done
labels_lvl1 = [0, 0, 0, 0] + [e for e in range(1, len(col_level1)) for _ in (0, 1)]
labels_lvl2 = range(len(col_level2))
cleaned.columns \
    = pd.MultiIndex(levels=[col_level1, col_level2], labels=[labels_lvl1, labels_lvl2])
cleaned = cleaned.set_index(('','name'))
cleaned.index.name = 'name'
cleaned.head()

### Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?
We start by correlating the related variables of the two rankings (e.g. QS rank with THE rank, QS faculty with THE faculty). This allows us to see if the two rankings are consistent, or if they have significant differences.

In [ ]:
cols_qs = merged_df[['rank_qs', 'faculty_total_qs', 'students_international_qs',
                  'students_total_qs', 'faculty_students_ratio_qs', 'international_ratio_qs']]
cols_the = merged_df[['rank_the', 'faculty_total_the', 'students_international_the',
                  'students_total_the', 'faculty_students_ratio_the', 'international_ratio_the']]
cols_qs.columns = cols_qs.columns.map(lambda x: x.rstrip('_qs'))
cols_the.columns = cols_the.columns.map(lambda x: x.rstrip('_the'))
corr = pd.DataFrame(cols_qs.corrwith(cols_the))
corr.index.name = 'Variable'
corr.columns = ['Correlation']
corr

We can see that the variables have a moderate correlation, but not a very strong one (except for *de facto* variables such as the number of students). This means that the two rankings use different criteria for evaluating a university.

In order to discover further information, we plot the full correlation matrix of our merged table.

In [ ]:
corr = merged_df.corr()
plt.figure(figsize=(10,10))
_ = sns.heatmap(corr, annot=True,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
# Show the variables with the highest absolute correlation
corr_pairs = abs(corr).unstack().reset_index()
corr_pairs.columns = [['var1', 'var2', 'corr']]
corr_pairs = corr_pairs[corr_pairs['var1'] > corr_pairs['var2']] # Remove redundant entries
corr_pairs.sort_values('corr', ascending=False).head(10)

We cannot find significant correlations between variables, apart from obvious ones. However, it seems that the QS rank is slightly correlated with the number of international faculty (0.49) and with the faculty/students ratio (0.42). This observation is not reflected in the THE ranking.

### Can you find the best university taking in consideration both rankings? Explain your approach.
We decided to take into account only the ranks for our analysis, for these reasons:
- The rank is a summarized indicator of all the other variables that could be taken into consideration for determining the best university, including those that are not visible in our dataset.
- If we used multiple variables, it would be difficult to determine which weights assign to each of them. In this case, we would create our own ranking, with arbitrary criteria.

As a first approach, we try to use to so-called Skyline operator, which is based on comparisons between pairs of tuples. The Skyline operator compares all pairs of tuples and removes those that are dominated by other tuples (are worse). The criterion for exclusion is based on Pareto optimality: a tuple is better than another if it is < in a variable and <= in all other variables. This approach is useful because it does not assign weights to fields (all variables are treated equal), but it is not guaranteed to return a single result.

In [ ]:
def dominates(t1, t2):
    if t1['rank_qs'] <= t2['rank_qs'] and t1['rank_the'] < t2['rank_the']:
        return True
    if t1['rank_qs'] < t2['rank_qs'] and t1['rank_the'] <= t2['rank_the']:
        return True
    return False

df = merged_df.copy()
dominates_all = np.ones(len(df), dtype=bool) # True if the i-th tuple dominates all other tuples
for i, t1 in df.iterrows():
    if dominates_all[i]:
        for j, t2 in df.iterrows():
            if dominates(t1, t2):
                dominates_all[j] = False
            
best_universities = df.loc[dominates_all]
best_universities

According to the Skyline operator, the above 4 universities are equivalent, as they are incomparable among each other (one of them is better in the QS ranking, and the other is better in the THE ranking).

In order to resolve ties, we decided to adopt the simplest approach: we sum the two ranks and select the university with the lowest sum.

In [ ]:
rankings = best_universities[['name', 'rank_qs', 'rank_the']].copy()
rankings['rank_avg'] = rankings['rank_qs'] + rankings['rank_the']
rankings.sort_values('rank_avg', ascending=True).head(10)

It appears that the best university is Stanford.